In [3]:
#import sasmodels
import sys
#sys.path.append('/home/huatthart/sasview/src')
sys.path.append('../sasview/src')
import numpy as np
from sasmodels.data import load_data
import matplotlib.pyplot as plt
import matplotlib
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
#from geomstats.geometry.functions import SRVF
from scipy.spatial import distance
from scipy.stats import gmean, hmean
from bumps.names import Parameter
from bumps.fitters import fit
import sasmodels
from sasmodels.core import load_model
from sasmodels.bumps_model import Model, Experiment
import bumps
import sys
#uncomment and edit line below to add path to the sasview source code
#sys.path.append("/path/to/sasview/src")
import sas
import os
import pickle
plt.rcParams.update({'font.size': 20})

## Load Data

In [4]:
filenames = os.listdir('../SAXS Structural Exploration/SAXS/Data/')
filenames

['A01.dat',
 'A02.dat',
 'A03.dat',
 'A04.dat',
 'A05.dat',
 'A06.dat',
 'A07.dat',
 'A08.dat',
 'A09.dat',
 'A10.dat',
 'A11.dat',
 'A12.dat',
 'B01.dat',
 'B02.dat',
 'B03.dat',
 'B04.dat',
 'B05.dat',
 'B06.dat',
 'B07.dat',
 'B08.dat',
 'B09.dat',
 'B10.dat',
 'B11.dat',
 'B12.dat',
 'C01.dat',
 'C02.dat',
 'C03.dat',
 'C04.dat',
 'C05.dat',
 'C06.dat',
 'C07.dat',
 'C08.dat',
 'C09.dat',
 'C10.dat',
 'C11.dat',
 'C12.dat',
 'D01.dat',
 'D02.dat',
 'D03.dat',
 'D04.dat',
 'D05.dat',
 'D06.dat',
 'D07.dat',
 'D08.dat',
 'D09.dat',
 'D10.dat',
 'D11.dat',
 'D12.dat',
 'E01.dat',
 'E02.dat',
 'E03.dat',
 'E04.dat',
 'E05.dat',
 'E06.dat',
 'E07.dat',
 'E08.dat',
 'E09.dat',
 'E10.dat',
 'E11.dat',
 'E12.dat',
 'F01.dat',
 'F02.dat',
 'F03.dat',
 'F04.dat',
 'F05.dat',
 'F06.dat',
 'F07.dat',
 'F08.dat',
 'F09.dat',
 'F10.dat',
 'F11.dat',
 'F12.dat',
 'G01.dat',
 'G02.dat',
 'G03.dat',
 'G04.dat',
 'G05.dat',
 'G06.dat',
 'G07.dat',
 'G08.dat',
 'G09.dat',
 'G10.dat',
 'G11.dat',
 'G1

## Choose Model 

In [12]:
kernel = sasmodels.core.load_model('cylinder+sphere')
model = sasmodels.bumps_model.Model(model=kernel)

In [22]:
model.parameters()

{'scale': Parameter(scale),
 'background': Parameter(background),
 'sld': Parameter(sld),
 'sld_solvent': Parameter(sld_solvent),
 'radius': Parameter(radius),
 'radius_pd': Parameter(radius_pd),
 'radius_pd_n': Parameter(radius_pd_n),
 'radius_pd_nsigma': Parameter(radius_pd_nsigma),
 'length': Parameter(length),
 'length_pd': Parameter(length_pd),
 'length_pd_n': Parameter(length_pd_n),
 'length_pd_nsigma': Parameter(length_pd_nsigma),
 'theta': Parameter(theta),
 'theta_pd': Parameter(theta_pd),
 'theta_pd_n': Parameter(theta_pd_n),
 'theta_pd_nsigma': Parameter(theta_pd_nsigma),
 'phi': Parameter(phi),
 'phi_pd': Parameter(phi_pd),
 'phi_pd_n': Parameter(phi_pd_n),
 'phi_pd_nsigma': Parameter(phi_pd_nsigma),
 'up_frac_i': Parameter(up_frac_i),
 'up_frac_f': Parameter(up_frac_f),
 'up_angle': Parameter(up_angle),
 'up_phi': Parameter(up_phi),
 'sld_M0': Parameter(sld_M0),
 'sld_mtheta': Parameter(sld_mtheta),
 'sld_mphi': Parameter(sld_mphi),
 'sld_solvent_M0': Parameter(sld_solvent

## Fit the Data in a Loop

In [30]:
chisq = []
mse = []
for i in range(len(filenames)):
    data = load_data('../SAXS Structural Exploration/SAXS/Data/' + filenames[i])
    data.x = data.x[data.x < 0.09]
    data.y = data.y[0:len(data.x)]
    data.dx = data.dx[0:len(data.x)]
    data.dy = data.dy[0:len(data.x)]
    data.mask = data.mask[0:len(data.x)]
    kernel = sasmodels.core.load_model('cylinder+sphere')
    
    #scale = Parameter(1, name='scale') # setting our fitting range to (0,1)
    background = Parameter(0, limits=(0,999), name='incoherent background')
#     scale = Parameter(0.001, limits=(0,999), name='scale').range(0,0.01)
#     sld = Parameter(4, limits=(0,999), name='sld') 
#     sld_solvent = Parameter(1, limits=(0,999), name='sld_solvent')
#     radius = Parameter(300, limits=(0,999), name='radius').range(0,600) 
#     length = Parameter(100, limits=(0,999), name='length').range(0,300) 
#     radius_pd = Parameter(0.5, limits=(0,999), name='radius_pd').range(0,300)
#     #radius_pd_n = Parameter(2, limits=(0,999), name='radius_pd_n').range(1,100)
#     #radius_pd_nsigma = Parameter(1, limits=(0,999), name = 'radius_pd_nsigma').range(1,100)
    
    #Cylinder
    scale = Parameter(1, limits=(0,999), name='scale')
    A_scale = Parameter(0.001, limits=(0,999), name='A_scale').range(0,0.01)
    A_sld = Parameter(4, limits=(0,999), name='A_sld') 
    A_sld_solvent = Parameter(1, limits=(0,999), name='A_sld_solvent')
    A_radius = Parameter(300, limits=(0,999), name='A_radius').range(0,600) 
    A_length = Parameter(100, limits=(0,999), name='A_length').range(0,300)
    A_radius_pd = Parameter(0.1, limits=(0,999), name='A_radius_pd').range(0,1)
    #radius_pd_n = Parameter(2, limits=(0,999), name='radius_pd_n')
    #A_radius_pd_nsigma = Parameter(10, limits=(0,999), name='A_radius_pd_nsigma')
    A_radius_pd_type = 'lognormal'
    #Sphere
    B_scale = Parameter(0.001, limits=(0,999), name='B_scale').range(0,0.01)
    B_sld = Parameter(4, limits=(0,999), name='B_sld') 
    B_sld_solvent = Parameter(1, limits=(0,999), name='B_sld_solvent')
    B_radius = Parameter(150, limits=(0,999), name='B_radius').range(0,400) 

    model = Model(model=kernel,
                  scale = scale,
                  background = background,
                  A_scale = A_scale,
                  A_sld = A_sld,
                  A_sld_solvent = A_sld_solvent,
                  A_radius = A_radius,
                  A_radius_pd = A_radius_pd,
                  #A_radius_pd_nsigma = A_radius_pd_nsigma, 
                  A_radius_pd_type = A_radius_pd_type,
                  A_length = A_length,
                  B_scale = B_scale,
                  B_sld = B_sld,
                  B_sld_solvent = B_sld_solvent,
                  B_radius = B_radius)
#     model = Model(model=kernel,
#                       scale = scale,
#                       background = background,
#                       sld = sld,
#                       sld_solvent = sld_solvent,
#                       radius = radius,
#                       length = length,
#                       #radius_pd_n = radius_pd_n,
#                       #radius_pd_nsigma = radius_pd_nsigma
#                      )
    experiment = sasmodels.bumps_model.Experiment(data=data, model=model)
    problem = bumps.fitproblem.FitProblem(experiment)
    results = bumps.fitters.fit(problem, method='lm', verbose=True)
    fig, ax = plt.subplots()
    problem.plot()
    matplotlib.rcParams.update({'font.size': 36})
    fig, ax = plt.subplots(figsize=(10,10))
    plt.scatter(data.x, data.y, s = 50, label = 'Data')
    plt.plot(data.x, experiment.theory(), color = 'red', label = 'Model', linewidth = 5)
    plt.yscale('log')
    plt.xscale('log')
    plt.xlabel('q (${\AA}^{-1}$)')
    plt.ylabel('Intensity (a.u.)')
    #plt.title('Sphere 25 nm in Radius')
    plt.legend()
    #plt.savefig('../Figures/230526_cylinder_sphere_polydisperse/' + filenames[i] + '.png', bbox_inches="tight")
    plt.close()
    plt.scatter(data.x, problem.residuals())
    plt.xlabel('q (${\AA}^{-1}$)')
    plt.ylabel('Residual')
    #plt.savefig('../Figures/230526_cylinder_sphere_polydisperse_residuals/' + filenames[i] + '.png', bbox_inches="tight")
    plt.close()
    mserror = np.mean(np.abs(np.log10(data.y) - np.log10(experiment.theory())))
    chisq.append(problem.chisq())
    mse.append(mserror)
    #np.save('../Merged_Data/SilverNP/230526/Residuals/' + filenames[i] + '.npy', problem.residuals())
    if i == 0:
        results_all = results.x.reshape(1,-1)
    else:
        results_all = np.vstack((results_all, results.x.reshape(1,-1)))

step 1 cost 165702.071(53)
                                A_length ...|......        100 in (0,300)
                                A_radius ....|.....        300 in (0,600)
                             A_radius_pd |.........        0.1 in (0,1)
                                 A_scale |.........      0.001 in (0,0.01)
                                B_radius ...|......        150 in (0,400)
                                 B_scale |.........      0.001 in (0,0.01)
step 2 cost 139.864(53)
step 3 cost 36.776(53)
step 4 cost 12.932(53)
step 5 cost 11.131(53)
step 6 cost 11.059(53)
step 7 cost 11.020(53)
step 8 cost 10.968(53)
step 9 cost 10.967(53)
step 10 cost 10.945(53)
step 11 cost 10.943(53)
step 12 cost 10.942(53)
step 13 cost 10.940(53)
step 14 cost 10.939(53)
step 15 cost 10.939(53)
step 16 cost 10.939(53)
step 17 cost 10.938(53)
step 18 cost 10.938(53)
step 19 cost 10.938(53)
step 20 cost 10.938(53)
step 21 cost 10.938(53)
step 22 cost 10.938(53)
step 23 cost 10.938(53)
step 24 

step 1 cost 137083.580(49)
                                A_length ...|......        100 in (0,300)
                                A_radius ....|.....        300 in (0,600)
                             A_radius_pd |.........        0.1 in (0,1)
                                 A_scale |.........      0.001 in (0,0.01)
                                B_radius ...|......        150 in (0,400)
                                 B_scale |.........      0.001 in (0,0.01)
step 2 cost 141.621(49)
step 3 cost 31.328(49)
step 4 cost 5.907(49)
step 5 cost 4.281(49)
step 7 cost 3.476(49)
step 8 cost 3.008(49)
step 9 cost 2.717(49)
step 10 cost 2.705(49)
step 11 cost 2.704(49)
step 12 cost 2.704(49)
step 13 cost 2.704(49)
step 14 cost 2.704(49)
step 15 cost 2.704(49)
step 16 cost 2.704(49)
final chisq 1.352(49)
=== Uncertainty from curvature:     name   value(unc.) ===
                                A_length   40.2(15)       
                                A_radius   137(11)        
            

step 8 cost 4.271(50)
step 9 cost 4.271(50)
step 10 cost 4.268(50)
step 11 cost 4.264(50)
step 12 cost 4.264(50)
step 13 cost 4.264(50)
step 14 cost 4.264(50)
step 15 cost 4.264(50)
step 16 cost 4.264(50)
step 17 cost 4.264(50)
final chisq 2.132(50)
=== Uncertainty from curvature:     name   value(unc.) ===
                                A_length   47.72(79)      
                                A_radius   219.8(90)      
                             A_radius_pd   0.207(52)      
                                 A_scale   185.5(21)e-6   
                                B_radius   134.6(12)      
                                 B_scale   41.2(16)e-6    


In [31]:
results_all

array([[4.72609886e+01, 2.19410736e+02, 2.60328273e-01, 2.01992826e-04,
        1.36000320e+02, 4.00192131e-05],
       [3.53664462e+01, 1.10096488e+02, 5.75537053e-01, 1.63474222e-04,
        7.85282182e+01, 2.06508806e-05],
       [3.66849440e+01, 1.11921484e+02, 3.99173566e-01, 1.82927675e-04,
        7.68869552e+01, 2.68480510e-05],
       [3.41357099e+01, 1.09596409e+02, 6.10258092e-01, 1.76389971e-04,
        8.73852409e+01, 2.55978121e-05],
       [3.36051446e+01, 1.24181907e+02, 5.35773663e-01, 2.84699560e-04,
        7.93393550e+01, 2.85966279e-05],
       [4.35588658e+01, 1.24329398e+02, 4.86547628e-01, 1.56371640e-04,
        8.70047331e+01, 2.06273057e-05],
       [4.02381556e+01, 1.36828822e+02, 3.05599079e-01, 1.55752592e-04,
        8.16724567e+01, 2.03694590e-05],
       [4.41734049e+01, 2.30081686e+02, 2.51596650e-01, 1.12996487e-04,
        9.21569767e+01, 1.35523659e-05],
       [4.61499287e+01, 1.86006087e+02, 3.61546235e-01, 1.67957287e-04,
        1.15338041e+02, 

In [32]:
results_all = np.hstack((np.array(filenames).reshape(-1,1), results_all))

In [33]:
df = pd.DataFrame(results_all, columns=['name', 'P_length','P_radius', 'P_radius_PD', 'P_scale','S_radius','S_scale'])
df

,name,P_length,P_radius,P_radius_PD,P_scale,S_radius,S_scale
0,A03.dat,47.2609886335662,219.41073564685314,0.2603282734902662,0.00020199282592171935,136.0003204193244,4.0019213100717564e-05
1,C04.dat,35.36644624972853,110.09648768526348,0.575537052789935,0.00016347422208787935,78.52821822179948,2.065088056620649e-05
2,D08.dat,36.68494400000979,111.92148396144012,0.3991735661836794,0.0001829276747405682,76.88695524222999,2.6848050976698602e-05
3,D09.dat,34.13570990761195,109.59640923590392,0.6102580923041,0.0001763899713579699,87.385240878186,2.5597812090291924e-05
4,D12.dat,33.605144614969994,124.18190698036996,0.5357736634706962,0.00028469955974062925,79.33935502882991,2.8596627876753474e-05
5,E01.dat,43.558865782657314,124.32939808987044,0.48654762752564584,0.00015637164033465236,87.00473309042329,2.0627305712867328e-05
6,E03.dat,40.23815558757618,136.82882155757542,0.3055990794757086,0.00015575259155106027,81.67245672335785,2.0369459032265934e-05
7,F12.dat,44.17340489046404,230.08168642582442,0.2515966499905647,0.00011299648742185823,92.15697670766725,1.3552365871922345e-05
8,T2A02.dat,46.14992870315632,186.0060867406408,0.3615462349239697,0.00016795728665805983,115.33804113716499,2.222326619218357e-05
9,T2B03.dat,27.298663557965643,140.8885472170027,0.3881297590304685,0.00017529406056533898,75.10605976082763,1.9939895594583097e-05


In [34]:
df.to_csv('../Figures/230526_cylinder_sphere_polydisperse/Cylinder_Sphere_Polydisperse_fit.csv')